In [181]:
import pandas as pd
import re

In [182]:
df = pd.read_csv("/home/rando149/shared/data/Collection_3165_Supporting_Documentation/abcd-tabulated-data-release-5.1/core/substance-use/su_y_sui.csv",low_memory=False)
df

,src_subject_id,eventname,su_today,tlfb_age,tlfb_age_month,tlfb_age_calc_inmonths,tlfb_alc,tlfb_tob,tlfb_mj,tlfb_mj_synth,...,xskipout_tob_l,xskipout_mj_l,xskipout_other_l,xskipout_end,xskipout_vape,xskipout_remote___1,xskipout_remote___2,xskipout_remote___3,xskipout_remote___4,xskipout_session
0,NDAR_INV003RTV85,baseline_year_1_arm_1,2018-10-01,10.0,11.0,131.0,1.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NDAR_INV003RTV85,1_year_follow_up_y_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NDAR_INV003RTV85,2_year_follow_up_y_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NDAR_INV003RTV85,3_year_follow_up_y_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NDAR_INV005V6D2C,baseline_year_1_arm_1,2018-04-22,10.0,1.0,121.0,1.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49146,NDAR_INVZZZP87KR,baseline_year_1_arm_1,NaN,10.0,7.0,127.0,1.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49147,NDAR_INVZZZP87KR,1_year_follow_up_y_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49148,NDAR_INVZZZP87KR,2_year_follow_up_y_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49149,NDAR_INVZZZP87KR,3_year_follow_up_y_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [183]:
filtered_df = df[df['eventname'] == '6_year_follow_up_y_arm_1']
other_columns = df.columns.drop('eventname')

for column in other_columns:
    counts = filtered_df[column].value_counts()
    print(f"Counts for {column}:")
    print(counts)
    print()

Counts for src_subject_id:
Series([], Name: count, dtype: int64)

Counts for su_today:
Series([], Name: count, dtype: int64)

Counts for tlfb_age:
Series([], Name: count, dtype: int64)

Counts for tlfb_age_month:
Series([], Name: count, dtype: int64)

Counts for tlfb_age_calc_inmonths:
Series([], Name: count, dtype: int64)

Counts for tlfb_alc:
Series([], Name: count, dtype: int64)

Counts for tlfb_tob:
Series([], Name: count, dtype: int64)

Counts for tlfb_mj:
Series([], Name: count, dtype: int64)

Counts for tlfb_mj_synth:
Series([], Name: count, dtype: int64)

Counts for tlfb_bitta:
Series([], Name: count, dtype: int64)

Counts for tlfb_caff:
Series([], Name: count, dtype: int64)

Counts for tlfb_inhalant:
Series([], Name: count, dtype: int64)

Counts for tlfb_rx_misuse:
Series([], Name: count, dtype: int64)

Counts for tlfb_list_yes_no:
Series([], Name: count, dtype: int64)

Counts for tlfb_list___1:
Series([], Name: count, dtype: int64)

Counts for tlfb_list___2:
Series([], Name: 

In [184]:
df.dtypes

src_subject_id          object
eventname               object
su_today                object
tlfb_age               float64
tlfb_age_month         float64
                        ...   
xskipout_remote___1    float64
xskipout_remote___2    float64
xskipout_remote___3    float64
xskipout_remote___4    float64
xskipout_session       float64
Length: 962, dtype: object

In [166]:
#function to choose which session to evaluate for which metric
def eval_metric(df, session, metric):
    su_df = df.copy()
    result_df = pd.DataFrame()
    new_df = pd.DataFrame()

    sessions = session.split(',')

    if isinstance(metric, list):
        if all(isinstance(sublist, list) for sublist in metric):
            metrics = [item for sublist in metric for item in sublist]
        else:
            metrics = metric
    else:
        metrics = [metric]

    for s in sessions:
        if s == '1':
            temp_df = su_df[su_df['eventname'].str.startswith('1_year')].copy()
            temp_df.loc[:, 'eventname'] = temp_df['eventname'].apply(lambda x: re.search('1_year_follow_up_y_arm_1', x).group() if re.search('1_year_follow_up_y_arm_1', x) else x)
            result_df = pd.concat([result_df, temp_df])
        elif s == '2':
            temp_df = su_df[su_df['eventname'].str.startswith('2_year')].copy()
            temp_df.loc[:, 'eventname'] = temp_df['eventname'].apply(lambda x: re.search('2_year_follow_up_y_arm_1', x).group() if re.search('2_year_follow_up_y_arm_1', x) else x)
            result_df = pd.concat([result_df, temp_df])
        elif s == '3':
            temp_df = su_df[su_df['eventname'].str.startswith('3_year')].copy()
            temp_df.loc[:, 'eventname'] = temp_df['eventname'].apply(lambda x: re.search('3_year_follow_up_y_arm_1', x).group() if re.search('3_year_follow_up_y_arm_1', x) else x)
            result_df = pd.concat([result_df, temp_df])
        elif s == '4':
            temp_df = su_df[su_df['eventname'].str.startswith('4_year')].copy()
            temp_df.loc[:, 'eventname'] = temp_df['eventname'].apply(lambda x: re.search('4_year_follow_up_y_arm_1', x).group() if re.search('4_year_follow_up_y_arm_1', x) else x)
            result_df = pd.concat([result_df, temp_df])
        elif s == 'baseline':
            temp_df = su_df[su_df['eventname'].str.startswith('baseline')].copy()
            temp_df.loc[:, 'eventname'] = temp_df['eventname'].apply(lambda x: re.search('baseline_year_1_arm_1', x).group() if re.search('baseline_year_1_arm_1', x) else x)
            result_df = pd.concat([result_df, temp_df])
        else:
            print(f"error with session: {s}")

    #testing results
    if not result_df.empty:
        print(result_df.head(5))
        new_df = result_df[['src_subject_id', 'eventname']].copy()
        print(new_df.head(5))
        for m in metrics:
            print(m)
            if m and m.lower() in result_df.columns:
                temp2_df = result_df[['src_subject_id', 'eventname', m.lower()]].copy()
                new_df = new_df.merge(temp2_df, on=['src_subject_id', 'eventname'], how='outer')
            elif m:
                print(f"\nWarning: metric '{m}' not found in input df")
            else:
                print("\nNo metric provided. returning all columns")
    else:
        print('error with sessions')

    return new_df

In [168]:
heard_of_l_columns = ['tlfb_alc_l',
                      'tlfb_tob_l',
                      'tlfb_mj_l',
                      'tlfb_mj_synth_l', #synthetic mj
                      'tlfb_bitta_l', #bittamugen or byphoditin
                      'tlfb_inhalant_l', #sniffing prodects to get high
                      'tlfb_rx_misuse_l', #prescription med abuse
                      'tlfb_list_yes_no_l'] #Have you heard of people using anything else to make them feel high, dizzy or different?

experimentation_columns = ['tlfb_alc_sip_l',
                           'tlfb_tob_puff_l',
                           'tlfb_mj_puff_l',
                           'tlfb_mj_synth_use_l', #synthetic mj
                           'tlfb_bitta_use_l', #bittamugen or byphoditin
                           'tlfb_inhalant_use_l', #sniffing prodects to get high
                           #'tlfb_sniff_use_l',
                           'tlfb_cough_use_l', #prescription med abuse
                           'tlfb_tranq_use_l',
                           'tlfb_vicodin_use_l',
                           'tlfb_opi_use_l',
                           'tlfb_coc_use_l', #cocaine
                           'tlfb_bsalts_use_l', #bath salts
                           'tlfb_meth_use_l', #methamphetamine
                           'tlfb_amp_use_l', #amphetamine
                           'tlfb_mdma_use_l',
                           'tlfb_ket_use_l', #ketamine
                           'tlfb_ghb_use_l', #gbh
                           'tlfb_shrooms_use_l', #mushrooms
                           'tlfb_salvia_use_l', #salvia
                           'tlfb_steroids_use_l'] #steroids

initiation_columns = ['tlfb_alc_use_l', #alcohol
                      'isip_2_l',
                      'tlfb_cig_use_l', #nicotine
                      'tlfb_ecig_use_l',
                      'tlfb_chew_use_l',
                      'tlfb_cigar_use_l',
                      'tlfb_hookah_use_l',
                      'tlfb_pipes_use_l',
                      'tlfb_mj_use_l', #marijuana
                      'tlfb_blunt_use_l',
                      'tlfb_edible_use_l',
                      'tlfb_mj_conc_use_l',
                      'tlfb_mj_drink_use_l',
                      'tlfb_tincture_use_l']
                      #'tlfb_mj_synth_use_l', #synthetic mj
                      #'tlfb_bitta_use_l', #bittamugen or byphoditin
                      #'tlfb_inhalant_use_l', #sniffing prodects to get high
                      #'tlfb_sniff_use_l',
                      #'tlfb_cough_use_l', #prescription med abuse
                      #'tlfb_tranq_use_l',
                      #'tlfb_vicodin_use_l',
                      #'tlfb_opi_use_l',
                      #'tlfb_coc_use_l', #cocaine
                      #'tlfb_bsalts_use_l', #bath salts
                      #'tlfb_meth_use_l', #methamphetamine
                      #'tlfb_amp_use_l', #amphetamine
                      #'tlfb_mdma_use_l',
                      #'tlfb_ket_use_l', #ketamine
                      #'tlfb_ghb_use_l', #gbh
                      #'tlfb_shrooms_use_l', #mushrooms
                      #'tlfb_salvia_use_l', #salvia
                      #'tlfb_steroids_use_l'] #steroids
                           
                           
                           
                           

metric_options = {
    "heard_of_l_columns": heard_of_l_columns,
    "experimentation_columns": experimentation_columns,
    "initiation_columns": initiation_columns
}

In [169]:
print(experimentation_columns)

['tlfb_alc_sip_l', 'tlfb_tob_puff_l', 'tlfb_mj_puff_l', 'tlfb_mj_synth_use_l', 'tlfb_bitta_use_l', 'tlfb_inhalant_use_l', 'tlfb_cough_use_l', 'tlfb_tranq_use_l', 'tlfb_vicodin_use_l', 'tlfb_opi_use_l', 'tlfb_coc_use_l', 'tlfb_bsalts_use_l', 'tlfb_meth_use_l', 'tlfb_amp_use_l', 'tlfb_mdma_use_l', 'tlfb_ket_use_l', 'tlfb_ghb_use_l', 'tlfb_shrooms_use_l', 'tlfb_salvia_use_l', 'tlfb_steroids_use_l']


In [170]:
session = input("Enter comma seperated list of follow up session to evaluate (ex: baseline,1,2,3,4): ")
metric_input = input("Enter variable name of interest, i.e. column name (ex: ): ")

Enter comma seperated list of follow up session to evaluate (ex: baseline,1,2,3,4):  baseline,1,2,3,4
Enter variable name of interest, i.e. column name (ex: ):  heard_of_l_columns,experimentation_columns,initiation_columns


In [171]:
metric_names = [name.strip() for name in metric_input.split(',')]
print(metric_names)
metrics = []

for name in metric_names:
    if name in metric_options:
        metrics.append(metric_options[name])
    else:
        print(f"Invalid metric option: {name}")

if metrics:
    eval_metric_df = eval_metric(df, session, metrics)
else:
    print("No valid metrics provided")

['heard_of_l_columns', 'experimentation_columns', 'initiation_columns']
      src_subject_id              eventname    su_today  tlfb_age  \
0   NDAR_INV003RTV85  baseline_year_1_arm_1  2018-10-01      10.0   
4   NDAR_INV005V6D2C  baseline_year_1_arm_1  2018-04-22      10.0   
8   NDAR_INV007W6H7B  baseline_year_1_arm_1         NaN      10.0   
11  NDAR_INV00BD7VDC  baseline_year_1_arm_1  2018-06-12       9.0   
14  NDAR_INV00CY2MDM  baseline_year_1_arm_1         NaN      10.0   

    tlfb_age_month  tlfb_age_calc_inmonths  tlfb_alc  tlfb_tob  tlfb_mj  \
0             11.0                   131.0       1.0       1.0      0.0   
4              1.0                   121.0       1.0       1.0      0.0   
8              6.0                   126.0       1.0       1.0      0.0   
11             5.0                   113.0       1.0       1.0      0.0   
14            10.0                   130.0       1.0       1.0      1.0   

    tlfb_mj_synth  ...  xskipout_tob_l  xskipout_mj_l  xskipou

In [173]:
eval_metric_df

,src_subject_id,eventname,tlfb_alc_l,tlfb_tob_l,tlfb_mj_l,tlfb_mj_synth_l,tlfb_bitta_l,tlfb_inhalant_l,tlfb_rx_misuse_l,tlfb_list_yes_no_l,...,tlfb_chew_use_l,tlfb_cigar_use_l,tlfb_hookah_use_l,tlfb_pipes_use_l,tlfb_mj_use_l,tlfb_blunt_use_l,tlfb_edible_use_l,tlfb_mj_conc_use_l,tlfb_mj_drink_use_l,tlfb_tincture_use_l
0,NDAR_INV003RTV85,1_year_follow_up_y_arm_1,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1,NDAR_INV003RTV85,2_year_follow_up_y_arm_1,NaN,NaN,NaN,0.0,0.0,1.0,1.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NDAR_INV003RTV85,3_year_follow_up_y_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NDAR_INV003RTV85,baseline_year_1_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NDAR_INV005V6D2C,1_year_follow_up_y_arm_1,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49146,NDAR_INVZZZP87KR,1_year_follow_up_y_arm_1,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
49147,NDAR_INVZZZP87KR,2_year_follow_up_y_arm_1,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
49148,NDAR_INVZZZP87KR,3_year_follow_up_y_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
49149,NDAR_INVZZZP87KR,4_year_follow_up_y_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [174]:
eval_metric_df['tlfb_coc_use_l'].value_counts(dropna=False)

tlfb_coc_use_l
NaN    31385
0.0    17765
1.0        1
Name: count, dtype: int64

In [175]:
eval_metric_df['tlfb_coc_use_l'].isna().sum()

31385

In [176]:
eval_metric_df['eventname'].isna().sum()

0

In [177]:
eval_metric_df['src_subject_id'].isna().sum()

0

In [178]:
cleaned_metric_df = eval_metric_df
#cleaned_metric_df = eval_metric_df.dropna(subset=heard_of_l_columns)

In [179]:
cleaned_metric_df

,src_subject_id,eventname,tlfb_alc_l,tlfb_tob_l,tlfb_mj_l,tlfb_mj_synth_l,tlfb_bitta_l,tlfb_inhalant_l,tlfb_rx_misuse_l,tlfb_list_yes_no_l,...,tlfb_chew_use_l,tlfb_cigar_use_l,tlfb_hookah_use_l,tlfb_pipes_use_l,tlfb_mj_use_l,tlfb_blunt_use_l,tlfb_edible_use_l,tlfb_mj_conc_use_l,tlfb_mj_drink_use_l,tlfb_tincture_use_l
0,NDAR_INV003RTV85,1_year_follow_up_y_arm_1,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1,NDAR_INV003RTV85,2_year_follow_up_y_arm_1,NaN,NaN,NaN,0.0,0.0,1.0,1.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NDAR_INV003RTV85,3_year_follow_up_y_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NDAR_INV003RTV85,baseline_year_1_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NDAR_INV005V6D2C,1_year_follow_up_y_arm_1,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49146,NDAR_INVZZZP87KR,1_year_follow_up_y_arm_1,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
49147,NDAR_INVZZZP87KR,2_year_follow_up_y_arm_1,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
49148,NDAR_INVZZZP87KR,3_year_follow_up_y_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
49149,NDAR_INVZZZP87KR,4_year_follow_up_y_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [180]:
cleaned_metric_df.to_csv('all_sui_l_metrics.csv', index=False)

In [144]:
event_order = ['baseline_year_1_arm_1', '1_year_follow_up_y_arm_1', '2_year_follow_up_y_arm_1', '3_year_follow_up_y_arm_1', '4_year_follow_up_y_arm_1']
cleaned_metric_df['eventname_ranked'] = cleaned_metric_df['eventname'].map({event: i for i, event in enumerate(event_order)})
cleaned_metric_df

,src_subject_id,eventname,tlfb_alc_l,tlfb_tob_l,tlfb_mj_l,tlfb_mj_synth_l,tlfb_bitta_l,tlfb_inhalant_l,tlfb_rx_misuse_l,tlfb_list_yes_no_l,...,tlfb_bsalts_use_l,tlfb_meth_use_l,tlfb_amp_use_l,tlfb_mdma_use_l,tlfb_ket_use_l,tlfb_ghb_use_l,tlfb_shrooms_use_l,tlfb_salvia_use_l,tlfb_steroids_use_l,eventname_ranked
0,NDAR_INV003RTV85,1_year_follow_up_y_arm_1,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,NDAR_INV003RTV85,2_year_follow_up_y_arm_1,NaN,NaN,NaN,0.0,0.0,1.0,1.0,0.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2
2,NDAR_INV003RTV85,3_year_follow_up_y_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,NDAR_INV003RTV85,baseline_year_1_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,NDAR_INV005V6D2C,1_year_follow_up_y_arm_1,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49146,NDAR_INVZZZP87KR,1_year_follow_up_y_arm_1,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
49147,NDAR_INVZZZP87KR,2_year_follow_up_y_arm_1,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
49148,NDAR_INVZZZP87KR,3_year_follow_up_y_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
49149,NDAR_INVZZZP87KR,4_year_follow_up_y_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4


In [145]:
metric_df_event_sorted = cleaned_metric_df.sort_values(['src_subject_id', 'eventname_ranked'], ascending=[True, False])
metric_df_event_sorted

,src_subject_id,eventname,tlfb_alc_l,tlfb_tob_l,tlfb_mj_l,tlfb_mj_synth_l,tlfb_bitta_l,tlfb_inhalant_l,tlfb_rx_misuse_l,tlfb_list_yes_no_l,...,tlfb_bsalts_use_l,tlfb_meth_use_l,tlfb_amp_use_l,tlfb_mdma_use_l,tlfb_ket_use_l,tlfb_ghb_use_l,tlfb_shrooms_use_l,tlfb_salvia_use_l,tlfb_steroids_use_l,eventname_ranked
2,NDAR_INV003RTV85,3_year_follow_up_y_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,NDAR_INV003RTV85,2_year_follow_up_y_arm_1,NaN,NaN,NaN,0.0,0.0,1.0,1.0,0.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2
0,NDAR_INV003RTV85,1_year_follow_up_y_arm_1,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,NDAR_INV003RTV85,baseline_year_1_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
6,NDAR_INV005V6D2C,3_year_follow_up_y_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49149,NDAR_INVZZZP87KR,4_year_follow_up_y_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
49148,NDAR_INVZZZP87KR,3_year_follow_up_y_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
49147,NDAR_INVZZZP87KR,2_year_follow_up_y_arm_1,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
49146,NDAR_INVZZZP87KR,1_year_follow_up_y_arm_1,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [146]:
def keep_latest_with_info(df, id_column, event_column, info_columns):
    # Function to check if a row has any non-NaN values in info columns
    def has_info(group):
        return group[info_columns].notna().any(axis=1)
    # Group by id, apply the has_info function, and keep the first True result
    mask = df.groupby(id_column).apply(has_info).reset_index(level=0, drop=True)
    # Return the filtered dataframe
    return df[mask].groupby(id_column).first().reset_index()
# Usage
metric_df_latest_sub_info = keep_latest_with_info(metric_df_event_sorted, 'src_subject_id', 'eventname', heard_of_l_columns)

/tmp/ipykernel_3359358/1116481029.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mask = df.groupby(id_column).apply(has_info).reset_index(level=0, drop=True)


In [147]:
print(metric_df_event_sorted.head())
print(metric_df_event_sorted.dtypes)

     src_subject_id                 eventname  tlfb_alc_l  tlfb_tob_l  \
2  NDAR_INV003RTV85  3_year_follow_up_y_arm_1         NaN         NaN   
1  NDAR_INV003RTV85  2_year_follow_up_y_arm_1         NaN         NaN   
0  NDAR_INV003RTV85  1_year_follow_up_y_arm_1         NaN         NaN   
3  NDAR_INV003RTV85     baseline_year_1_arm_1         NaN         NaN   
6  NDAR_INV005V6D2C  3_year_follow_up_y_arm_1         NaN         NaN   

   tlfb_mj_l  tlfb_mj_synth_l  tlfb_bitta_l  tlfb_inhalant_l  \
2        NaN              NaN           NaN              NaN   
1        NaN              0.0           0.0              1.0   
0        1.0              0.0           0.0              0.0   
3        NaN              NaN           NaN              NaN   
6        NaN              NaN           NaN              NaN   

   tlfb_rx_misuse_l  tlfb_list_yes_no_l  ...  tlfb_bsalts_use_l  \
2               NaN                 NaN  ...                0.0   
1               1.0                 0.0  .

In [148]:
metric_df = metric_df_latest_sub_info.drop(['eventname','eventname_ranked'], axis=1)
metric_df

,src_subject_id,tlfb_alc_l,tlfb_tob_l,tlfb_mj_l,tlfb_mj_synth_l,tlfb_bitta_l,tlfb_inhalant_l,tlfb_rx_misuse_l,tlfb_list_yes_no_l,tlfb_alc_sip_l,...,tlfb_coc_use_l,tlfb_bsalts_use_l,tlfb_meth_use_l,tlfb_amp_use_l,tlfb_mdma_use_l,tlfb_ket_use_l,tlfb_ghb_use_l,tlfb_shrooms_use_l,tlfb_salvia_use_l,tlfb_steroids_use_l
0,NDAR_INV003RTV85,NaN,NaN,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,NDAR_INV005V6D2C,NaN,NaN,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NDAR_INV00BD7VDC,NaN,NaN,1.0,NaN,0.0,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NDAR_INV00CY2MDM,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.0,0.0,...,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,NDAR_INV00HEV6HB,NaN,NaN,NaN,1.0,0.0,1.0,NaN,1.0,0.0,...,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11453,NDAR_INVZZLZCKAY,NaN,NaN,NaN,1.0,0.0,1.0,NaN,0.0,0.0,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
11454,NDAR_INVZZNX6W2P,NaN,NaN,NaN,0.0,0.0,1.0,1.0,0.0,0.0,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
11455,NDAR_INVZZPKBDAC,NaN,NaN,NaN,1.0,0.0,NaN,1.0,0.0,0.0,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
11456,NDAR_INVZZZ2ALR6,NaN,NaN,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
metric_df.to_csv('subjectID_plus_mostrecent_heardof_metrics.csv', index=False)